In [2]:
import pickle as pkl 

In [3]:
with open("results_wikiBio_llama.pickle", "rb") as f:
    pred = pkl.load(f)

In [6]:
import json 

with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gemini/wikibio/top_tables_wiki.json", "r") as f:
    data = json.load(f)
    

In [7]:
gt_tables_dict = {}
for tables in data:
    for k, v in tables.items():
        gt_tables_dict[k] = v

In [9]:
pred_tables = {}
for k in pred:
    for i, j in k.items():
        pred_tables[i] = j

In [10]:
len(gt_tables_dict), len(pred_tables)

(2000, 2000)

In [13]:
import re
import pandas as pd
def extract_table(raw_text):
    """
    Extracts a table from a raw text string which may be in one of several formats:
    - Plain text with code fences (e.g., starting with ```text and ending with ```)
    - A JSON string that contains a "result" key with a table string, possibly wrapped in triple quotes.
    
    The table rows are expected to be separated by the literal "<NEWLINE>" token,
    and each row is a pipe-separated list of fields.
    """
    text = raw_text.strip()
    
    # --- Step 1. If the input is JSON, extract the "result" field ---
    try:
        parsed = json.loads(text)
        if isinstance(parsed, dict) and "result" in parsed:
            text = parsed["result"]
    except json.JSONDecodeError:
        # Not valid JSON, so assume it's a plain string
        pass

    # --- Step 2. Remove markdown code fences if present ---
    # Remove starting fence like ```text (or any language) and the ending ```
    text = re.sub(r'^```(?:[^\n]*)\n', '', text)
    text = re.sub(r'\n```$', '', text)

    # --- Step 3. Remove extra triple quotes if present ---
    text = text.strip()
    if text.startswith('"""') and text.endswith('"""'):
        text = text[3:-3].strip()

    # --- Step 4. Split the string into rows using the <NEWLINE> token ---
    if "<NEWLINE>" in text:
        rows = text.split("<NEWLINE>")
    else:
        rows = text.split("\n")
    records = []
    for row in rows:
        row = row.strip()
        if not row:
            continue
        # Remove any leading/trailing pipes and split on "|"
        # Then filter out any empty strings after stripping whitespace
        parts = [p.strip() for p in row.strip('|').split("|") if p.strip()]
        if len(parts) >= 2:
            key = parts[0]
            # In case there are additional tokens, join them with " | " to preserve inner separators.
            value = " | ".join(parts[1:])
            records.append((key, value))
    
    # --- Step 5. Create a DataFrame from the records ---
    df = pd.DataFrame(records, columns=["Field", "Value"])
    return df

# --- Example 1: Table wrapped in a markdown code fence ---
raw_data2 = """```text\n| name | john \"jack\" reynolds | <NEWLINE> | birth date | 21 february 1869 | <NEWLINE> | death date | 12 march 1917 | <NEWLINE> | clubs | west bromwich albion, aston villa, celtic, southampton | <NEWLINE> | international appearances | ireland (5), england (8) | <NEWLINE> | honours | FA Cup (1892 with West Bromwich Albion), English League titles (3 with Aston Villa), FA Cups (2 with Aston Villa) | <NEWLINE> | death place | sheffield |\n```\n"""
# --- Example 2: JSON string containing the table in the "result" key ---
raw_data1 = """| death place | sheffield, england | <NEWLINE> | name | jack reynolds | <NEWLINE> | death date | 12 march 1917 | <NEWLINE> | birth date | 21 february 1869 | <NEWLINE> | fullname | john reynolds |"""

# --- Parse and display the tables ---
df1 = extract_table(raw_data1)
df2 = extract_table(raw_data2)

print("Table from Example 1:")
print(df1.to_markdown(index=False))
print("\nTable from Example 2:")
print(df2.to_markdown(index=False))


Table from Example 1:
| Field       | Value              |
|:------------|:-------------------|
| death place | sheffield, england |
| name        | jack reynolds      |
| death date  | 12 march 1917      |
| birth date  | 21 february 1869   |
| fullname    | john reynolds      |

Table from Example 2:
| Field                     | Value                                                                                                             |
|:--------------------------|:------------------------------------------------------------------------------------------------------------------|
| name                      | john "jack" reynolds                                                                                              |
| birth date                | 21 february 1869                                                                                                  |
| death date                | 12 march 1917                                                                     

In [16]:
input_tables = {}
for k, v in gt_tables_dict.items():
    input_tables[k] = [extract_table(v).to_markdown(index= False), extract_table(pred_tables[k]).to_markdown(index= False)]

In [25]:
json.dump(input_tables, open("input_tables_for_score.json", "w"), indent= 4)

In [26]:
print(input_tables["38008"][0])

| Field         | Value                                                                         |
|:--------------|:------------------------------------------------------------------------------|
| office        | member of the south carolina house of representatives from spartanburg county |
| rank          | captain                                                                       |
| birth place   | spartanburg, south carolina                                                   |
| resting place | spartanburg, south carolina                                                   |
| state         | south carolina                                                                |
| term end      | march 3, 1921                                                                 |
| branch        | south carolina national guard                                                 |
| alma mater    | wofford college virginia polytechnic institute university of chicago          |
| predecessor   | jo

In [27]:
print(input_tables["38008"][1])

| Field        | Value                                                                                              |
|:-------------|:---------------------------------------------------------------------------------------------------|
| birth place  | spartanburg, south carolina                                                                        |
| death date   | november 23, 1937                                                                                  |
| name         | samuel jones nicholls                                                                              |
| birth date   | may 7, 1885                                                                                        |
| occupation   | united states representative, lawyer                                                               |
| death place  | spartanburg, south carolina                                                                        |
| residence    | south carolina                         

In [20]:
input_tables.keys()

dict_keys(['41536', '5325', '43823', '38008', '34967', '39863', '43653', '67856', '55424', '27670', '58162', '11950', '9989', '9894', '26281', '30985', '55350', '40585', '5321', '33912', '50486', '62462', '5001', '55254', '27010', '31640', '58753', '33066', '37405', '33883', '67942', '15243', '58170', '67430', '27632', '45067', '49639', '52586', '66175', '43046', '11318', '16411', '18060', '50655', '61108', '9140', '61164', '3219', '71029', '4702', '59467', '10460', '55907', '65942', '19340', '67976', '526', '58128', '35267', '21334', '71559', '49299', '71347', '55732', '12145', '25707', '46465', '69927', '70387', '71953', '13023', '12114', '28870', '53028', '71152', '55327', '63033', '3406', '38249', '54615', '55546', '24391', '33079', '26324', '71180', '23599', '68468', '69206', '36162', '44910', '10741', '53281', '26583', '17150', '57578', '45932', '39711', '57206', '15789', '66643', '45933', '11860', '25889', '47013', '22247', '25196', '15425', '53086', '25989', '28121', '18638', '